In [1]:
# general packages
import numpy as np
import pandas as pd
import datatable as dt
import gc

## Data preposessing

In [2]:
%%time
#parse_dates = ['CreatedDate', 'RequestedDate', 'AccTime', 'PupTime', 'DelTime']
#A = pd.read_csv('Booking_Dispatch_training_set.csv', parse_dates = parse_dates, 
#date_parser = lambda x: pd.to_datetime(x).strftime("%Y-%m-%d %H:%M:%S"))
#data_parsed = A.to_csv('data_parsed.csv')

Wall time: 0 ns


In [3]:
%%time
#parse_dates = ['CreatedDate', 'RequestedDate']
#B = pd.read_csv('Booking_Dispatch_training_set.csv', parse_dates = parse_dates, 
#date_parser = lambda x: pd.to_datetime(x).strftime("%Y-%m-%d %H:%M:%S"))
#data_parsed = B.to_csv('val_data_parsed.csv')

Wall time: 0 ns


In [4]:
%%time
train_data_datatable = dt.fread('data_parsed.csv') 
data = train_data_datatable.to_pandas()
data = data.iloc[:,1:]

Wall time: 2min 17s


In [5]:
%%time
val_data_datatable = dt.fread('val_data_parsed.csv')
val_data = val_data_datatable.to_pandas()
val_data = val_data.iloc[:,1:]

Wall time: 2min 25s


In [6]:
data.shape

(3000000, 37)

In [7]:
val_data.shape

(3000000, 37)

In [8]:
#subtract the features and target variable for modeling 
train_features = data[['BookingSource','Conditions',
     'HasCondition','IsAccountBooking','CreatedDate','RequestedDate',
     'BookingFleet','DispatchFleet','AreaNumber','PuPlace','PuAddress',
     'PuSuburb','PuLat','PuLong','TargetVariable']]

In [9]:
val_features = val_data[['BookingSource','Conditions',
     'HasCondition','IsAccountBooking','CreatedDate','RequestedDate',
     'BookingFleet','DispatchFleet','AreaNumber','PuPlace','PuAddress',
     'PuSuburb','PuLat','PuLong','TargetVariable']]

In [10]:
train_features.head()

,BookingSource,Conditions,HasCondition,IsAccountBooking,CreatedDate,RequestedDate,BookingFleet,DispatchFleet,AreaNumber,PuPlace,PuAddress,PuSuburb,PuLat,PuLong,TargetVariable
0,TelOp,0,No,No,2019-04-11 10:36:55,2019-04-11 11:00:00,13cabs Sydney,13cabs Sydney,177,House,11 Stewart St,Randwick,-33.921022,151.244202,Completed
1,TelOp,0,No,No,2019-06-23 09:52:55,2019-06-23 09:52:55,13cabs Sydney,13cabs Sydney,111,,145 Alfred Rd,Narraweena,-33.755931,151.273028,Cancelled
2,Internet,0,No,No,2019-12-01 00:14:02,2019-12-01 00:14:02,13cabs Sydney,13cabs Sydney,92,,284 Birrell Street,Bondi,-33.897201,151.261205,Completed
3,StratumIVR,0,No,No,2019-09-12 09:07:03,2019-09-12 09:07:04,13cabs Sydney,13cabs Sydney,182,,127-131 Cook Rd,Centennial Park,-33.895716,151.228609,Completed
4,Internet,0,No,No,2019-02-21 21:37:04,2019-02-21 21:37:04,13cabs Sydney,13cabs Sydney,4,,Unit 87/5010 Macquarie St,Sydney,-33.859737,151.213307,Cancelled


In [11]:
val_features.head()

,BookingSource,Conditions,HasCondition,IsAccountBooking,CreatedDate,RequestedDate,BookingFleet,DispatchFleet,AreaNumber,PuPlace,PuAddress,PuSuburb,PuLat,PuLong,TargetVariable
0,TelOp,0,No,No,2019-04-11 10:36:55,2019-04-11 11:00:00,13cabs Sydney,13cabs Sydney,177,House,11 Stewart St,Randwick,-33.921022,151.244202,Completed
1,TelOp,0,No,No,2019-06-23 09:52:55,2019-06-23 09:52:55,13cabs Sydney,13cabs Sydney,111,,145 Alfred Rd,Narraweena,-33.755931,151.273028,Cancelled
2,Internet,0,No,No,2019-12-01 00:14:02,2019-12-01 00:14:02,13cabs Sydney,13cabs Sydney,92,,284 Birrell Street,Bondi,-33.897201,151.261205,Completed
3,StratumIVR,0,No,No,2019-09-12 09:07:03,2019-09-12 09:07:04,13cabs Sydney,13cabs Sydney,182,,127-131 Cook Rd,Centennial Park,-33.895716,151.228609,Completed
4,Internet,0,No,No,2019-02-21 21:37:04,2019-02-21 21:37:04,13cabs Sydney,13cabs Sydney,4,,Unit 87/5010 Macquarie St,Sydney,-33.859737,151.213307,Cancelled


In [22]:
#concatenate training data and validation data for feature engineering
features = pd.concat([train_features, val_features], axis=0).reset_index(drop=True)

In [28]:
features

,BookingSource,Conditions,HasCondition,IsAccountBooking,CreatedDate,RequestedDate,BookingFleet,DispatchFleet,AreaNumber,PuPlace,PuAddress,PuSuburb,PuLat,PuLong,TargetVariable
0,TelOp,0,No,No,2019-04-11 10:36:55,2019-04-11 11:00:00,13cabs Sydney,13cabs Sydney,177,House,11 Stewart St,Randwick,-33.921022,151.244202,Completed
1,TelOp,0,No,No,2019-06-23 09:52:55,2019-06-23 09:52:55,13cabs Sydney,13cabs Sydney,111,,145 Alfred Rd,Narraweena,-33.755931,151.273028,Cancelled
2,Internet,0,No,No,2019-12-01 00:14:02,2019-12-01 00:14:02,13cabs Sydney,13cabs Sydney,92,,284 Birrell Street,Bondi,-33.897201,151.261205,Completed
3,StratumIVR,0,No,No,2019-09-12 09:07:03,2019-09-12 09:07:04,13cabs Sydney,13cabs Sydney,182,,127-131 Cook Rd,Centennial Park,-33.895716,151.228609,Completed
4,Internet,0,No,No,2019-02-21 21:37:04,2019-02-21 21:37:04,13cabs Sydney,13cabs Sydney,4,,Unit 87/5010 Macquarie St,Sydney,-33.859737,151.213307,Cancelled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,TelOp,0,No,No,2019-03-01 08:55:10,2019-03-01 09:30:00,13cabs Sydney,13cabs Sydney,209,Apartment,77-79 Lawrence St,Peakhurst,-33.960439,151.059493,Completed
5999996,TelOp,0,No,No,2019-07-12 19:47:53,2019-07-12 19:47:53,13cabs Sydney,Silver Service Sydney,401,Business,85 Dunning Ave,Rosebery,-33.916820,151.202503,Completed
5999997,TelOp,0,No,No,2019-08-15 15:03:46,2019-08-15 15:03:47,13cabs Sydney,13cabs Sydney,182,Sydney Cricket Ground Members,Driver Ave,Moore Park,-33.891001,151.223461,Completed
5999998,TelOp,0,No,No,2019-05-16 17:23:18,2019-05-16 18:30:00,13cabs Sydney,Silver Service Sydney,19,Milson Appts,50 Alfred St,Milsons Point,-33.849802,151.211508,Completed


In [24]:
# check datatypes
features.dtypes

BookingSource        object
Conditions            int64
HasCondition         object
IsAccountBooking     object
CreatedDate          object
RequestedDate        object
BookingFleet         object
DispatchFleet        object
AreaNumber            int32
PuPlace              object
PuAddress            object
PuSuburb             object
PuLat               float64
PuLong              float64
TargetVariable       object
dtype: object

In [25]:
#BookingSource variable
Conditions_count = data['Conditions'].value_counts().rename_axis('unique_values').reset_index(name='counts')
Conditions_count

,unique_values,counts
0,0,2240288
1,4194304,304956
2,64,101869
3,262144,88487
4,16777216,38880
...,...,...
375,537133056,1
376,604012544,1
377,17196908544,1
378,262152,1


In [26]:
feature1 = features

In [ ]:
features_parsed = feature1.to_csv('feature1_parsed.csv')

## Feature engineering

Wall time: 1min 47s


,Conditions,HasCondition,IsAccountBooking,CreatedDate,RequestedDate,BookingFleet,DispatchFleet,AreaNumber,PuPlace,PuAddress,PuSuburb,PuLat,PuLong,TargetVariable,Internet,Permanent,StratumIVR,TelOp
0,0,No,No,2019-04-11 10:36:55,2019-04-11 11:00:00,13cabs Sydney,13cabs Sydney,177,House,11 Stewart St,Randwick,-33.921022,151.244202,Completed,0,0,0,1
1,0,No,No,2019-06-23 09:52:55,2019-06-23 09:52:55,13cabs Sydney,13cabs Sydney,111,,145 Alfred Rd,Narraweena,-33.755931,151.273028,Cancelled,0,0,0,1
2,0,No,No,2019-12-01 00:14:02,2019-12-01 00:14:02,13cabs Sydney,13cabs Sydney,92,,284 Birrell Street,Bondi,-33.897201,151.261205,Completed,1,0,0,0
3,0,No,No,2019-09-12 09:07:03,2019-09-12 09:07:04,13cabs Sydney,13cabs Sydney,182,,127-131 Cook Rd,Centennial Park,-33.895716,151.228609,Completed,0,0,1,0
4,0,No,No,2019-02-21 21:37:04,2019-02-21 21:37:04,13cabs Sydney,13cabs Sydney,4,,Unit 87/5010 Macquarie St,Sydney,-33.859737,151.213307,Cancelled,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,0,No,No,2019-03-01 08:55:10,2019-03-01 09:30:00,13cabs Sydney,13cabs Sydney,209,Apartment,77-79 Lawrence St,Peakhurst,-33.960439,151.059493,Completed,0,0,0,1
5999996,0,No,No,2019-07-12 19:47:53,2019-07-12 19:47:53,13cabs Sydney,Silver Service Sydney,401,Business,85 Dunning Ave,Rosebery,-33.916820,151.202503,Completed,0,0,0,1
5999997,0,No,No,2019-08-15 15:03:46,2019-08-15 15:03:47,13cabs Sydney,13cabs Sydney,182,Sydney Cricket Ground Members,Driver Ave,Moore Park,-33.891001,151.223461,Completed,0,0,0,1
5999998,0,No,No,2019-05-16 17:23:18,2019-05-16 18:30:00,13cabs Sydney,Silver Service Sydney,19,Milson Appts,50 Alfred St,Milsons Point,-33.849802,151.211508,Completed,0,0,0,1


### Create new time features

In [31]:
# Changing the pickup_datetime and dropoff_datetime from object to datetime datatype
feature1['CreatedDate']=pd.to_datetime(feature1['CreatedDate'])
feature1['RequestedDate']=pd.to_datetime(feature1['RequestedDate'])

In [32]:
# Creating features based on day
feature1['create_by_day'] = feature1['CreatedDate'].dt.day_name()
feature1['request_by_day'] = feature1['RequestedDate'].dt.day_name()

In [33]:
# Creating features based on Hour
feature1['create_by_hour'] = feature1['CreatedDate'].dt.hour
feature1['request_by_hour'] = feature1['RequestedDate'].dt.hour
#Morning, which starts at 6:01 am and ends at 12:oopm
#Afternoon, which starts at 12:01 pm and ends at 18:00pm
#Evening, which starts at 18:01 and ends at 21:00pm
#Night, which start at 21:01 and ends at 6:00am

In [34]:
def part_of_day (t):
    if t in range (6,12):
        return "Morning"
    elif t in range (12,18):
        return "Afternoon"
    elif t in range (18,21):
        return "Evening"
    else:
        return "Night"

In [36]:
feature1['create_part_of_day']=feature1['create_by_hour'].apply(part_of_day)
feature1['request_part_of_day']=feature1['request_by_hour'].apply(part_of_day)

In [38]:
feature1.head()

,Conditions,HasCondition,IsAccountBooking,CreatedDate,RequestedDate,BookingFleet,DispatchFleet,AreaNumber,PuPlace,PuAddress,...,Internet,Permanent,StratumIVR,TelOp,create_by_day,request_by_day,create_by_hour,request_by_hour,create_part_of_day,request_part_of_day
0,0,No,No,2019-04-11 10:36:55,2019-04-11 11:00:00,13cabs Sydney,13cabs Sydney,177,House,11 Stewart St,...,0,0,0,1,Thursday,Thursday,10,11,Morning,Morning
1,0,No,No,2019-06-23 09:52:55,2019-06-23 09:52:55,13cabs Sydney,13cabs Sydney,111,,145 Alfred Rd,...,0,0,0,1,Sunday,Sunday,9,9,Morning,Morning
2,0,No,No,2019-12-01 00:14:02,2019-12-01 00:14:02,13cabs Sydney,13cabs Sydney,92,,284 Birrell Street,...,1,0,0,0,Sunday,Sunday,0,0,Night,Night
3,0,No,No,2019-09-12 09:07:03,2019-09-12 09:07:04,13cabs Sydney,13cabs Sydney,182,,127-131 Cook Rd,...,0,0,1,0,Thursday,Thursday,9,9,Morning,Morning
4,0,No,No,2019-02-21 21:37:04,2019-02-21 21:37:04,13cabs Sydney,13cabs Sydney,4,,Unit 87/5010 Macquarie St,...,1,0,0,0,Thursday,Thursday,21,21,Night,Night


### new feature: if booking fleet matches with the dispatch fleet

### New feature: if the records of pick up surburbs exceed 10 

In [51]:
feature1['PuSuburb'].value_counts().head(1000)

Sydney             206150
Mascot             146598
Randwick           121972
Mosman              93100
Liverpool           90028
                    ...  
SYDENHAM               12
LINLEY POINT           12
WATSONS BAY            12
Megalong Valley        12
LEUMEAH                10
Name: PuSuburb, Length: 1000, dtype: int64

In [52]:
pick = feature1['PuSuburb'].value_counts().tail(1000).keys().tolist() #<10
pick

['Green Valley',
 'Wentworthville',
 'Riverwood',
 'Bella Vista',
 'Kellyville',
 'Rydalmere',
 'Bass Hill',
 'Putney',
 'Condell Park',
 'Wetherill Park',
 'Kingswood',
 'Rosehill',
 'Tamarama',
 'Katoomba',
 'Chiswick',
 'Padstow',
 'Beverly Hills',
 'Mount Colah',
 'Bradbury',
 'Hazelbrook',
 'Chester Hill',
 'McMahons Point',
 'Kurraba Point',
 'Brookvale',
 'Mortdale',
 'Lavender Bay',
 'St Johns Park',
 'Seaforth',
 'Longueville',
 'Daceyville',
 'Leppington',
 'Bossley Park',
 'Heckenberg',
 'Cabramatta West',
 'Toongabbie',
 'Chifley',
 'Darling Harbour',
 'Strathfield South',
 'Hinchinbrook',
 'West Pymble',
 'Mount Druitt',
 'East Lindfield',
 'Leura',
 'Prairiewood',
 'Beecroft',
 'Penshurst',
 'Silverwater',
 'Rooty Hill',
 'Forestville',
 'Breakfast Point',
 'Balgowlah',
 'Sans Souci',
 'Sylvania',
 'Quakers Hill',
 'Greystanes',
 'Fairfield Heights',
 'Carlton',
 'Panania',
 'St. Ives',
 'Northmead',
 'Ambarvale',
 'St Marys',
 'Fairfield West',
 'Normanhurst',
 'Lalor Pa

In [53]:
feature1['pick'] = np.where(feature1['PuSuburb'].isin(pick), True, False)

In [54]:
feature1.head()

,Conditions,HasCondition,IsAccountBooking,BookingFleet,DispatchFleet,AreaNumber,PuPlace,PuAddress,PuSuburb,PuLat,...,Permanent,StratumIVR,TelOp,create_by_day,request_by_day,create_by_hour,request_by_hour,create_part_of_day,request_part_of_day,pick
0,0,No,No,13cabs Sydney,13cabs Sydney,177,House,11 Stewart St,Randwick,-33.921022,...,0,0,1,Thursday,Thursday,10,11,Morning,Morning,False
1,0,No,No,13cabs Sydney,13cabs Sydney,111,,145 Alfred Rd,Narraweena,-33.755931,...,0,0,1,Sunday,Sunday,9,9,Morning,Morning,True
2,0,No,No,13cabs Sydney,13cabs Sydney,92,,284 Birrell Street,Bondi,-33.897201,...,0,0,0,Sunday,Sunday,0,0,Night,Night,False
3,0,No,No,13cabs Sydney,13cabs Sydney,182,,127-131 Cook Rd,Centennial Park,-33.895716,...,0,1,0,Thursday,Thursday,9,9,Morning,Morning,False
4,0,No,No,13cabs Sydney,13cabs Sydney,4,,Unit 87/5010 Macquarie St,Sydney,-33.859737,...,0,0,0,Thursday,Thursday,21,21,Night,Night,False


### Drop columns

In [ ]:
feature1 = feature1.drop(columns=['CreatedDate', 'RequestedDate']) #date features

In [ ]:
feature1 = feature1.drop(columns=['PuSuburb','PuPlace','PuAddress']) # pick-up location related features

In [ ]:
feature1 = feature1.drop(columns=['PuSuburb'])

### Creature dummy variables for categorical data

In [ ]:
%%time
# BookingSource 
dummies = pd.get_dummies(feature1['BookingSource'], drop_first=False)
feature1 = feature1.join(dummies)
feature1 = feature1.drop('BookingSource', axis= 1)

In [ ]:
feature1

## Modeling